# Mn-Salinity relationship


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4 as nc
import pandas as pd 
import glob
import warnings
warnings.filterwarnings('ignore')
import datetime
import xarray as xr

%matplotlib inline

#### Parameters:

In [ ]:
# domain dimensions:
imin, imax = 1479, 2179
jmin, jmax = 159, 799
isize = imax - imin
jsize = jmax - jmin

year  = 2015
month = 8
# Mn model results location and ANHA12 salinity (http://knossos.eas.ualberta.ca/anha/anhatable.php)
folder_salinity      = f'/data/brogalla/ANHA12/salinity_ave/'
folder_ref_2009      = f'/data/brogalla/run_storage/Mn-reference-202110/ANHA12_ref-2009_20211012/'
folder_ref_2015      = f'/data/brogalla/run_storage/Mn-reference-202110/ANHA12_ref-2015_20211012/'
folder_cleanice_2009 = f'/data/brogalla/run_storage/Mn-clean-ice-202110/ANHA12_clean-ice-2009_20211023/'
folder_cleanice_2015 = f'/data/brogalla/run_storage/Mn-clean-ice-202110/ANHA12_clean-ice-2015_20211023/'

# colours:
obs_CB     = '#b23333'
obs_CAA    = '#d89999'
mod_CB     = '#006f99'
mod_CAA    = '#99c5d6'
land_color = "#8b7765"

#### Load files:

In [ ]:
# ANHA12 grid:
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_lon   = np.array(mesh.variables['nav_lon'])
mesh_lat   = np.array(mesh.variables['nav_lat'])
mesh_bathy = np.array(mesh.variables['hdept'][0])
tmask      = np.array(mesh.variables['tmask'][0,:,:,:])
Z_masked   = np.ma.masked_where((tmask > 0.1), tmask) 

Model results

In [ ]:
# Mn model results format:
c   = nc.Dataset(f'{folder_ref_2009}ANHA12_EXH006_2009_monthly.nc', 'r')
lat_model      = np.array(c.variables['nav_lat'])
lon_model      = np.array(c.variables['nav_lon'])
depths_model   = np.array(c.variables['deptht'])

Observations

In [ ]:
# Concentration of dMn in 2015 from Colombo et al., 2020 
Manuel = pd.read_csv('/ocean/brogalla/GEOTRACES/data/DMn_nmolL-1_BR-edited.csv')
Manuel_station_names = Manuel['Station'].astype('string')
Manuel_depths        = Manuel['Depth'].astype('float').values
Manuel_dMn_nmolkg    = Manuel['nmol kg-1'].astype('float').values
Manuel_dMn_nmolL     = Manuel['nmol L-1'].astype('float')
Manuel_salinity      = Manuel['Salinity'].astype('float')
Manuel_density       = Manuel_dMn_nmolL / Manuel_dMn_nmolkg

# Fit of Mn:salinity from river observations in Colombo et al. 2019:
S_obs       = np.arange(0, 40, 1)
dmn_obs_CB  = -0.601*S_obs + 21.9 
dmn_obs_CAA = -1.26*S_obs + 45.2  

In [ ]:
# GEOTRACES stations information from Colombo et al., 2020
Pb_data   = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Pb-paper-data.csv')
stn_names = Pb_data['Station names'].dropna().astype('str')
lons      = Pb_data['station lon'].astype('float').dropna().values
lats      = Pb_data['Station lat'].astype('float').dropna().values

In [ ]:
# Concentration of dMn in the Beaufort Sea in 2009.
### Thesis doesn't have a list of lats and lons of station locations --- Jeffrey Charter's thesis does (2012)
IPY = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Nari_Sim_dMn_data.csv')
IPY_names_full = IPY['Station name'].astype('string').values
IPY_depths     = IPY['Depth [m]'].astype('float').values
IPY_dMn_nmolkg = IPY['DMn [ nmol/kg]'].astype('float').values
IPY_lons_full  = -1*IPY['Longitude'].astype('float').values
IPY_lats_full  = IPY['Latitude'].astype('float').values
IPY_salinity   = IPY['Salinity [psu]'].astype('float').values

# Unique entries (maintaining order)
IPY_lons  = np.array(list(dict.fromkeys(IPY_lons_full)))
IPY_lats  = np.array(list(dict.fromkeys(IPY_lats_full)))
IPY_names = np.array(list(dict.fromkeys(IPY_names_full)))

# Convert nmol / kg to nmol / L
IPY_density             = IPY['Density'].astype('float').values / 1000 # kg / L
IPY_depths_cruise_sheet = IPY['Depth [m] cruise sheet'].astype('float').values
IPY_dMn_nmolL = IPY_dMn_nmolkg * IPY_density

#### Define functions:

In [ ]:
def calculate_average_2009(folder_ref_2009, folder_salinity):
    # Calculate average over data files during cruise period:
    #-------------------------------------------------------------------------------------------------------------------
    # 2009 GEOTRACES: 29 August - 8 September
    dmn_files_2009  = np.sort(glob.glob(f'{folder_ref_2009}ANHA12_EXH006_5d_20090101_20091231*'))
    sal_files_2009  = np.sort(glob.glob(f'{folder_salinity}ANHA12-EXH006_5d_gridT_y2009m*'))
    start_date_2009 = datetime.datetime.strptime('2009-08-29', '%Y-%m-%d')
    end_date_2009   = datetime.datetime.strptime('2009-09-08', '%Y-%m-%d')

    #----Manganese---------------
    dmn_file_list_2009 = []
    for file in dmn_files_2009:
        if (datetime.datetime.strptime(file.split('/')[-1][42:50],'%Y%m%d') >= start_date_2009) & \
                (datetime.datetime.strptime(file.split('/')[-1][51:59],'%Y%m%d') < end_date_2009):
            dmn_file_list_2009.append(file)
    dmn_2009 = np.zeros((50,700,640))
    days = 0
    for file in dmn_file_list_2009:
        ds = xr.open_dataset(f"{file}")
        dmn_2009 = dmn_2009 + ds['dissolmn'].values[0,:,:,:]
        
    mn_dis_ref_2009 = dmn_2009 / len(dmn_file_list_2009)
    
    #----Salinity---------------
    sal_file_list_2009 = []
    for file in sal_files_2009:
        if (datetime.datetime.strptime(file.split('/')[-1][24:34],'%Ym%md%d') >= start_date_2009) & \
                (datetime.datetime.strptime(file.split('/')[-1][24:34],'%Ym%md%d')+datetime.timedelta(days=5) < end_date_2009):
            sal_file_list_2009.append(file)
    sal_2009 = np.zeros((50,700,640))
    days = 0
    for file in sal_file_list_2009:
        ds = xr.open_dataset(f"{file}")
        sal_2009 = sal_2009 + ds['vosaline'].values[0,:,imin:imax,jmin:jmax]
        
    sal_ref_2009 = sal_2009 / len(sal_file_list_2009)
    
    return mn_dis_ref_2009, sal_ref_2009

In [ ]:
def calculate_average_2015(folder_ref_2015, folder_salinity):
    # 2015 GEOTRACES: 9 August - 15 September
    dmn_files_2015  = np.sort(glob.glob(f'{folder_ref_2015}ANHA12_EXH006_5d_20150101_20151231*'))
    sal_files_2015  = np.sort(glob.glob(f'{folder_salinity}ANHA12-EXH006_5d_gridT_y2015m*'))
    start_date_2015 = datetime.datetime.strptime(f'2015-08-09', '%Y-%m-%d')
    end_date_2015   = datetime.datetime.strptime(f'2015-09-15', '%Y-%m-%d')
    
    #----Manganese---------------
    dmn_file_list_2015 = []
    for file in dmn_files_2015:
        if (datetime.datetime.strptime(file.split('/')[-1][42:50],'%Y%m%d') >= start_date_2015) & \
                    (datetime.datetime.strptime(file.split('/')[-1][51:59],'%Y%m%d') < end_date_2015):
            dmn_file_list_2015.append(file)

    dmn_2015 = np.zeros((50,700,640))
    days = 0
    for file in dmn_file_list_2015:
        ds = xr.open_dataset(f"{file}")
        dmn_2015 = dmn_2015 + ds['dissolmn'].values[0,:,:,:]

    mn_dis_ref_2015 = dmn_2015 / len(dmn_file_list_2015)
    
    #----Salinity---------------
    sal_file_list_2015 = []
    for file in sal_files_2015:
        if (datetime.datetime.strptime(file.split('/')[-1][24:34],'%Ym%md%d') >= start_date_2015) & \
                (datetime.datetime.strptime(file.split('/')[-1][24:34],'%Ym%md%d')+datetime.timedelta(days=5) < end_date_2015):
            sal_file_list_2015.append(file)
    
    sal_2015 = np.zeros((50,700,640))
    days = 0
    for file in sal_file_list_2015:
        ds = xr.open_dataset(f"{file}")
        sal_2015 = sal_2015 + ds['vosaline'].values[0,:,imin:imax,jmin:jmax]    
        
    sal_ref_2015 = sal_2015 / len(sal_file_list_2015)
    
    return mn_dis_ref_2015, sal_ref_2015

In [ ]:
def mask_data(mask_ini, S_model, mn_model):
    # Apply a spatial mask to input data
    
    z_start = 0 #5:17 
    z_end   = 50 # Observation relationships are only for upper 40 m and start at 5 m
    mask    = np.tile(mask_ini, (z_end-z_start,1,1))
    S  = np.ma.masked_where(mask==0 , S_model[z_start:z_end,:,:])
    mn = np.ma.masked_where(mask==0 , mn_model[z_start:z_end,:,:]*10**9)
    
    S_plot  = S[:,:,:].filled(fill_value=np.nan).flatten()
    mn_plot = mn[:,:,:].filled(fill_value=np.nan).flatten()
    
    S_plot[S_plot < 0.1]   = np.nan
    mn_plot[mn_plot < 0.1] = np.nan

    return S_plot, mn_plot

In [ ]:
def plot_mnsal(S_CAA, mn_CAA, S_CB, mn_CB, reso=200, title='', savefig=False):
    
    fig, ax = plt.subplots(1,3, figsize=(4.5*2,3.74*0.7), dpi=300)
    st = sns.axes_style("whitegrid")

    for axis in ax.ravel():
        axis.set(xlim=(22, 36), ylim=(0, 20))
        axis.tick_params(axis='both', labelsize=7.5)

    with st:  
        CAA_color='#e58203' # '#e5ae03'
        level_step = 0.1
        linewidth_fit = 2.7
        sns.kdeplot(x=S_CAA[::reso], y=mn_CAA[::reso], levels=np.arange(level_step,1+level_step,level_step), \
                    fill=True, ax=ax[0], cmap="YlOrBr_r")
        sns.kdeplot(x=S_CB[::reso], y=mn_CB[::reso], levels=np.arange(level_step,1+level_step,level_step), \
                    fill=True, ax=ax[1], cmap='Blues_r')
        sns.kdeplot(x=S_CB[::reso], y=mn_CB[::reso], levels=np.arange(level_step,1+level_step,level_step), \
                    fill=True, ax=ax[2], cmap='Blues_r')

        # CAA
        sns.regplot(Manuel_salinity[Manuel_station_names.str.find('CAA')!=-1], \
                    Manuel_dMn_nmolL[Manuel_station_names.str.find('CAA')!=-1], \
                    dropna=True, ci=None, \
                    scatter_kws={"s": 15, 'color':'w', 'edgecolor':CAA_color, 'linewidths':1.0, 'alpha':1.0},\
                    line_kws={"linewidth":linewidth_fit, 'color':CAA_color, 'linestyle':'--'}, truncate=False, ax=ax[0], \
                    label='2015 observations')

        # Canada Basin
        sns.regplot(Manuel_salinity[Manuel_station_names.str.find('CB')!=-1], \
                    Manuel_dMn_nmolL[Manuel_station_names.str.find('CB')!=-1], \
                    dropna=True, ci=None, \
                    scatter_kws={"s": 15, 'color':'w', 'edgecolor':'#6f818d', 'linewidths':1.0, 'alpha':1.0},\
                    line_kws={"linewidth":0}, truncate=False, ax=ax[1], \
                    label='2015 observations')

        # IPY data
        sns.regplot(IPY_salinity, IPY_dMn_nmolL, \
                    dropna=True, ci=None, \
                    scatter_kws={"s": 15, 'color':'w', 'edgecolor':'#39454d', 'linewidths':1.0, 'alpha':1.0},\
                    line_kws={"linewidth":0}, truncate=False, ax=ax[1], \
                    label='2009 observations')


        # Observations combined fit
        sns.regplot(S_CB_obs_combined, mn_CB_obs_combined, \
                    dropna=True, ci=None, \
                    scatter_kws={"s": 0}, line_kws={"linewidth":linewidth_fit, 'color':'#607381', 'linestyle':'--'}, \
                    truncate=False, ax=ax[1])

        # Model data
        sns.regplot(S_CAA[::reso], mn_CAA[::reso], dropna=True, ci=None, \
                    line_kws={"linewidth":linewidth_fit, 'color':'w'}, scatter_kws={"s": 0.0}, ax=ax[0])
        sns.regplot(S_CAA[::reso], mn_CAA[::reso], dropna=True, ci=None, \
                    line_kws={"linewidth":2.0, 'color':CAA_color}, scatter_kws={"s": 0.0}, ax=ax[0])
        sns.regplot(S_CB[::reso] , mn_CB[::reso], dropna=True, ci=None,  \
                    line_kws={"linewidth":linewidth_fit, 'color':'w'}, scatter_kws={"s": 0.0}, ax=ax[1])
        sns.regplot(S_CB[::reso] , mn_CB[::reso], dropna=True, ci=None, \
                    line_kws={"linewidth":2.0, 'color':'#607381'}, scatter_kws={"s": 0.0}, ax=ax[1])

        # Legend entries:
        sns.lineplot([0,0],[0,0], lw=2, c=CAA_color, ax=ax[0], label='Model fit')
        sns.lineplot([0,0],[0,0], lw=2, c=CAA_color, ax=ax[0], label='Observations fit')
        ax[0].lines[4].set_linestyle("--")
        sns.lineplot([0,0],[0,0], lw=2, c='#607381', ax=ax[1], label='Model fit')
        sns.lineplot([0,0],[0,0], lw=2, c='#607381', ax=ax[1], label='Observations fit')
        ax[1].lines[6].set_linestyle("--")

        ax[0].legend(loc=(0.35,0.76), frameon=False, fontsize=7)
        ax[1].legend(loc=(0.35,0.68), frameon=False, fontsize=7)
        ax[0].set_xlabel('Salinity [psu]', fontsize=8)
        ax[1].set_xlabel('Salinity [psu]', fontsize=8)
        ax[2].set_xlabel('Salinity [psu]', fontsize=8)
        ax[0].set_ylabel('Dissolved Mn [nM]', fontsize=8)
        ax[1].set_ylabel('')
        ax[2].set_ylabel('')

        ax[0].set_title('CAA', fontsize=8)
        ax[1].set_title('Canada Basin', fontsize=8)
        ax[2].set_title('Canada Basin', fontsize=8)

    fig.text(0.07, 0.94, 'a)', fontsize=9)
    fig.text(0.35, 0.94, 'b)', fontsize=9)
    fig.text(0.65, 0.94, 'c)', fontsize=9)
#     fig.text(0.07, 0.94, 'd)', fontsize=9)
#     fig.text(0.35, 0.94, 'e)', fontsize=9)
#     fig.text(0.65, 0.94, 'f)', fontsize=9)
    
    if savefig:
        fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202110/S9-model-mn-salinity-relationship_reference.png', \
               bbox_inches='tight', dpi=300)
        fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202110/S9-model-mn-salinity-relationship_reference.svg', \
                       format='svg', bbox_inches='tight', dpi=300)
#         fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202110/S9-model-mn-salinity-relationship_clean-ice.png', \
#                bbox_inches='tight', dpi=300)
#         fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202110/S9-model-mn-salinity-relationship_clean-ice.svg', \
#                        format='svg', bbox_inches='tight', dpi=300)
    return

Calculate average Mn concentrations during time period of interest

In [ ]:
mn_ref_2009, sal_2009 = calculate_average_2009(folder_ref_2009, folder_salinity)
mn_ref_2015, sal_2015 = calculate_average_2015(folder_ref_2015, folder_salinity)

mn_cleanice_2009, sal_2009 = calculate_average_2009(folder_cleanice_2009, folder_salinity)
mn_cleanice_2015, sal_2015 = calculate_average_2015(folder_cleanice_2015, folder_salinity)

# Salinity:Mn relationship

Seperate Canada Basin and CAA

In [ ]:
# Canada Basin: ------
x_ind = np.arange(1621, 2100, 1)
y_ind = (-7/8)*x_ind + 1517 + 700

CB_indx = []
CB_indy = []

for index in range(0,len(x_ind)):
    CB_x = np.arange(x_ind[index],2179,1)
    CB_y = np.ones(CB_x.shape)*y_ind[index]
    
    CB_indx = np.append(CB_x, CB_indx)
    CB_indy = np.append(CB_y, CB_indy)
    
# Baffin Bay: ------
x_ind = np.arange(1400, 1550, 1)
y_ind = np.arange(515, 799, 1)

BB_indx = []
BB_indy = []
for xindex in x_ind:
    for yindex in y_ind:        
        BB_indx = np.append(BB_indx, xindex)
        BB_indy = np.append(BB_indy, yindex)
        
# Separate Canada Basin and the CAA: -------
mask_ini_CB   = np.zeros((isize,jsize))
mask_ini_CAA  = np.ones((isize,jsize))
mask_ini_CBBB = np.zeros((isize,jsize))

for i, j in zip(CB_indx, CB_indy):
    mask_ini_CB[int(i-imin),int(j-jmin)]   = 1
    mask_ini_CBBB[int(i-imin),int(j-jmin)] = 1
    mask_ini_CAA[int(i-imin),int(j-jmin)]  = 0
    
for i, j in zip(BB_indx, BB_indy):
    mask_ini_CBBB[int(i-imin),int(j-jmin)] = 1
    mask_ini_CAA[int(i-imin),int(j-jmin)]  = 0
    
mask_ini_CB[150:-1 ,-8:-1]   = 1
mask_ini_CBBB[150:-1 ,-8:-1] = 1
mask_ini_CAA[150:-1,-8:-1] = 0

Separate salinity and mn results based on these definitons.

In [ ]:
reso = 5
ref_S_CB_2015, ref_mn_CB_2015   = mask_data(mask_ini_CBBB[::reso,::reso], sal_2015[:,::reso,::reso], mn_ref_2015[:,::reso,::reso])
ref_S_CB_2009, ref_mn_CB_2009   = mask_data(mask_ini_CBBB[::reso,::reso], sal_2009[:,::reso,::reso], mn_ref_2009[:,::reso,::reso])
ref_S_CAA_2015, ref_mn_CAA_2015 = mask_data(mask_ini_CAA[::reso,::reso], sal_2015[:,::reso,::reso], mn_ref_2015[:,::reso,::reso])

cleanice_S_CB_2015, cleanice_mn_CB_2015   = mask_data(mask_ini_CB[::reso,::reso], sal_2015[:,::reso,::reso], \
                                                      mn_cleanice_2015[:,::reso,::reso])
cleanice_S_CB_2009, cleanice_mn_CB_2009   = mask_data(mask_ini_CB[::reso,::reso], sal_2009[:,::reso,::reso], \
                                                      mn_cleanice_2009[:,::reso,::reso])
cleanice_S_CAA_2015, cleanice_mn_CAA_2015 = mask_data(mask_ini_CAA[::reso,::reso], sal_2015[:,::reso,::reso], \
                                                      mn_cleanice_2015[:,::reso,::reso])

# Combine Canada Basin 2009 and 2015 model data:
ref_S_CB_combined       = np.append(ref_S_CB_2009, ref_S_CB_2015)
ref_mn_CB_combined      = np.append(ref_mn_CB_2009, ref_mn_CB_2015)
cleanice_S_CB_combined  = np.append(cleanice_S_CB_2009, cleanice_S_CB_2015)
cleanice_mn_CB_combined = np.append(cleanice_mn_CB_2009, cleanice_mn_CB_2015)

# Combine Canada Basin 2009 and 2015 observation data:
S_CB_obs_combined  = np.append(IPY_salinity, Manuel_salinity[Manuel_station_names.str.find('CB')!=-1])
mn_CB_obs_combined = np.append(IPY_dMn_nmolL, Manuel_dMn_nmolL[Manuel_station_names.str.find('CB')!=-1])

In [ ]:
plot_mnsal(ref_S_CAA_2015, ref_mn_CAA_2015, ref_S_CB_combined, ref_mn_CB_combined, \
           reso=1, title='', savefig=True)

In [ ]:
plot_mnsal(cleanice_S_CAA_2015, cleanice_mn_CAA_2015, cleanice_S_CB_combined, cleanice_mn_CB_combined, \
           reso=1, title='', savefig=True)